In [76]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset 
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [77]:
train_dataloader, test_dataloader, input_size, output_size = create_dataloaders_for_dataset(dataset_name='breast_cancer', task='classification', batch_size=32)

xtrain size 455
xtest size 114
y_train size 455
y_test size 114


In [78]:
class NN(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        hidden_size = input_size*100
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [79]:
multiLayerPerceptron = NN(input_size, output_size)

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp2b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=2,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [80]:
DTEmlp2b.train_DTE()

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 2 models
Model 1:


Training Process:   0%|          | 0/10 [00:00<?, ?it/s]

Training Process: 100%|██████████| 10/10 [00:00<00:00, 11.76it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 11.40it/s]


./saved_models/model_2.pth
Model 4:


Training Process: 100%|██████████| 10/10 [00:00<00:00, 13.08it/s]

./saved_models/model_4.pth
 Maximum accuracy: 0.9765625, F1: 0.9388674848032436, Test: 0.0656525855883956, Train: 0.06526061632360021 
 



,DTE_acc,DTE_f1,DTE_test,DTE_train
0,0.976562,0.938867,0.065653,0.065261


In [81]:
DTEmlp2b.single_model()

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process:   0%|          | 0/30 [00:00<?, ?it/s]

Training Process: 100%|██████████| 30/30 [00:02<00:00, 11.06it/s]

 Maximum accuracy: 0.9764178240740741, F1: 0.9699174493255733, Test: 0.07284503214371701, Train: 0.09153912527900603 
 



,single_acc,single_f1,single_test,single_train
0,0.976418,0.969917,0.072845,0.091539


In [87]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        print(f'Trail {i}: \n')
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [88]:
output_directory = './fixed.csv'
run_experiment(DTEmlp2b, trails=10, output_directory=output_directory)

Trail 0: 

|| TRAINING WEIGHT AVERAGING TREE ENSEMBLE ||
Task: classification
Randomly initializing 2 models
Model 1:


Training Process:   0%|          | 0/10 [00:00<?, ?it/s]

Training Process: 100%|██████████| 10/10 [00:00<00:00, 12.52it/s]


./saved_models/model_1.pth
Model 2:


Training Process: 100%|██████████| 10/10 [00:01<00:00,  9.72it/s]


...

./saved_models/model_2.pth
Model 4:


Training Process: 100%|██████████| 10/10 [00:01<00:00,  8.85it/s]


./saved_models/model_4.pth
 Maximum accuracy: 0.98203125, F1: 0.9378334412821995, Test: 0.06351097198203207, Train: 0.07020957816392183 
 

|| TRAINING SINGLE MODEL || 
Task: Classification


Training Process: 100%|██████████| 30/30 [00:03<00:00,  8.17it/s]

 Maximum accuracy: 0.9650607638888888, F1: 0.963407800965749, Test: 0.07667267271317542, Train: 0.0616927930381563 
 



In [89]:
df = pd.read_csv("fixed.csv")

In [90]:
print(df.columns)

Index(['DTE_acc', 'DTE_f1', 'DTE_test', 'DTE_train', 'single_acc', 'single_f1',
       'single_test', 'single_train'],
      dtype='object')


In [91]:
print(f'single acc mean: {df["single_acc"].mean()}  DTE acc mean: {df["DTE_acc"].mean()}')
print(f'single acc var: {df["single_acc"].var()}  DTE acc var: {df["DTE_acc"].var()}')
print(f'single loss mean: {df["single_test"].mean()}  DTE loss mean: {df["DTE_test"].mean()}')
print(f'single loss var: {df["single_test"].var()}  DTE loss var: {df["DTE_test"].var()}')

single acc mean: 0.9716076581115644  DTE acc mean: 0.9813411458333334
single acc var: 4.5940692159192304e-05  DTE acc var: 3.0218032815982836e-05
single loss mean: 0.07084112031385296  DTE loss mean: 0.06576444366015488
single loss var: 6.166525851852026e-05  DTE loss var: 9.398227766711837e-06


In [ ]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [ ]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)